<img src='./img/opengeohub_logo.png' alt='OpenGeoHub Logo' align='right' width='15%'></img>

<a href="./00_index.ipynb"><< Index</a><br>
<a href="./04_aws_open_data_registry.ipynb"><< 04 - AWS - Open Data Registry</a><span style="float:right;"><a href="./11_covid19_case_study_s5p_anomaly_map.ipynb">11 - Covid-19 case study - Sentinel-5P anomaly map>></a></span>

# Google Earth Engine

### About

<img src="./img/gee_logo_2.png" align="right" width="35%"></img>
[Google Earth Engine (GEE)](https://earthengine.google.com/) is a planetary-scale platform for Earth science data & analysis. 

There are several ways to work with the Google Earth Engine:
* [Code Editor]( code.earthengine.google.com), a web-based IDE in Javascript
* [Client libraries](https://github.com/google/earthengine-api) provide Javascript and Python wrapper functions for the Earth Engine API

You have to [sign up](https://signup.earthengine.google.com/) for GEE.

### Data

Earth Engine's data archive includes:
* **Weather and Climate Data**
    * A selection of ERA5 reanalysis
      * [ECMWF/ERA5/DAILY](https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_DAILY)
      * [ECMWF/ERA5/MONTHLY](https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_MONTHLY)
    * TRMM precipitation
* **Imagery**
    * Landsat
    * Sentinels, e.g
      * [Sentinel-2](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR)
      * [Sentinel-5P](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S5P_OFFL_L3_NO2)
      * [Sentinel-3](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S3_OLCI)
    * MODIS
    
... and many more. Have a look yourself at the [Earth Engine Data Catalog](https://developers.google.com/earth-engine/datasets/catalog/).

In case you are interested in uploading your own data in Google Earth Engine, see here a [workflow](https://github.com/jwagemann/era5_in_gee) how to upload ERA5 data, using [manifest uploads](https://developers.google.com/earth-engine/image_manifest). 

### How to retrieve data?

Required libraries:
* [Earth Engine Python API](https://developers.google.com/earth-engine/python_install)
* [geemap](https://geemap.readthedocs.io/en/latest/index.html)

<hr>

#### Load required libraries

In [3]:
import ee
import geemap

#### First step of the Earth Engine Python API is to run the `ee.authenticate()` command

The command will prompt you to open a given URL, which asks you to authenticate with Google Earth Engine with your email address you registered with Google Earth Engine. If the autentication was successful, the browser window provides you a code which you will be able to enter. 
This token is then saved and you are good to go to use the EarthEngine Python API.

In [5]:
ee.Authenticate()

Enter verification code: 4/3AGg_XO3C3gEqj_5yocp_gQUJ6qCmIPb5zy-KR1BI-D_vAX9jrCq1as

Successfully saved authorization token.


#### Initialize the EarthEngine Python API with the `ee.Initialize()` command

In [8]:
# Initialize ee with access token
ee.Initialize()

#### Load an ERA5/MONTHLY image and get image information

The `ee.Image` functions can be used to load a single image from a `ImageCollection` of the EarthEngine Data Catalog. With the `getInfo()` command, you can get a print-out of what your image contains. The print-out shows you that the Image contains nine different layers.

In [13]:
img_test = ee.Image('ECMWF/ERA5/MONTHLY/200001')
img_test.getInfo()

{'type': 'Image',
 'bands': [{'id': 'mean_2m_air_temperature',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [1440, 721],
   'crs': 'EPSG:4326',
   'crs_transform': [0.25, 0, -180, 0, -0.25, 90]},
  {'id': 'minimum_2m_air_temperature',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [1440, 721],
   'crs': 'EPSG:4326',
   'crs_transform': [0.25, 0, -180, 0, -0.25, 90]},
  {'id': 'maximum_2m_air_temperature',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [1440, 721],
   'crs': 'EPSG:4326',
   'crs_transform': [0.25, 0, -180, 0, -0.25, 90]},
  {'id': 'dewpoint_2m_temperature',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [1440, 721],
   'crs': 'EPSG:4326',
   'crs_transform': [0.25, 0, -180, 0, -0.25, 90]},
  {'id': 'total_precipitation',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [1440, 721],
   'crs': 'EPSG:4326',
   'crs_trans

#### Select one specific parameter

Now, let's select the two layers `mean_2m_air_temperature` and `total_precipitation` and print again more information about the two images with the `getInfo()` command. 

The `getInfo()` command shows you the dimension and associated crs of the variable and additional properties.

In [11]:
t2m = img_test.select('mean_2m_air_temperature')
tp = img_test.select('total_precipitation')
tp.getInfo()

{'type': 'Image',
 'bands': [{'id': 'total_precipitation',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [1440, 721],
   'crs': 'EPSG:4326',
   'crs_transform': [0.25, 0, -180, 0, -0.25, 90]}],
 'version': 1578581373737848,
 'id': 'ECMWF/ERA5/MONTHLY/200001',
 'properties': {'system:time_start': 946684800000,
  'month': 1,
  'year': 2000,
  'system:footprint': {'type': 'LinearRing',
   'coordinates': [[-180, -90],
    [180, -90],
    [180, 90],
    [-180, 90],
    [-180, -90]]},
  'system:time_end': 949363200000,
  'system:asset_size': 37702939,
  'system:index': '200001'}}

#### Get image url and apply visualization params to it

The `geemap` package is based on `ipyleaflet` and brings you the functionalities of the EarthEngine Javascript editor to Python.
You can create a `geemap` object that understand the EarthEngine commands to specify your map.

In [12]:
Map = geemap.Map(zoom=2,
                layout={'height':'500px'})
t2m_palette=['#000080','#0000D9','#4000FF','#8000FF','#0080FF','#00FFFF','#00FF80','#80FF00','#DAFF00','#FFFF00','#FFF500','#FFDA00','#FFB000','#FFA400','#FF4F00','#FF2500','#FF0A00','#FF00FF']
Map.addLayer(t2m, {'min': 250, 'max': 330, 'palette':t2m_palette})
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

<br>

<a href="./04_aws_open_data_registry.ipynb"><< 04 - AWS - Open Data Registry</a><span style="float:right;"><a href="./11_covid19_case_study_s5p_anomaly_map.ipynb">11 - Covid-19 case study - Sentinel-5P anomaly map>></a></span><br>
<a href="./00_index.ipynb"><< Index</a>

<hr>
&copy; 2020 | Julia Wagemann
<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img style="float: right" alt="Creative Commons Lizenzvertrag" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a>